In [355]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import prophet
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

In [356]:
dataset_path = '../data/'

pollen_test = pd.read_csv(dataset_path + 'new_pollen_test.csv')
pollen_val = pd.read_csv(dataset_path + 'new_pollen_val.csv')
pollen_train = pd.read_csv(dataset_path + 'new_pollen_train.csv')
pollen_type = pd.read_csv(dataset_path + 'pollen-type.csv')
submission_example = pd.read_csv(dataset_path + 'submission_example.csv')
weather_data = pd.read_csv(dataset_path + 'weather_data.csv')

In [357]:
print(max(pollen_train.AMBROSIA))

0.0


In [358]:
# make index column
pollen_train['date_idx'] = [i for i in range(len(pollen_train))]
pollen_val['date_idx'] = [i for i in range(len(pollen_val))]
pollen_test['date_idx'] = [i for i in range(len(pollen_test))]

In [359]:
pollen_train.drop(['date'], axis=1, inplace=True)
pollen_val.drop(['date'], axis=1, inplace=True)
pollen_test.drop(['date'], axis=1, inplace=True)

In [360]:
pollen_train.shape

(355, 27)

In [361]:
pollen_test.shape

(112, 28)

## Model

In [362]:
import warnings
import copy
warnings.filterwarnings('ignore')

In [363]:
columns = pollen_train.columns
batch_id = pollen_test['batch_id']

In [364]:
batch_id

0        1
1        2
2        3
3        4
4        5
      ... 
107    108
108    109
109    110
110    111
111    112
Name: batch_id, Length: 112, dtype: int64

In [365]:
ans_columns = pd.DataFrame(columns=['batch_id', '1 day prediction', '2 days prediction', '3 days prediction'])
ans_columns['batch_id'] = [val for val in batch_id]

In [372]:
pollen_test_copy = copy.deepcopy(pollen_test)

for idx, column in enumerate(columns):
    if column == 'date_idx':
        continue
    batch_id = pollen_test.drop(['batch_id'], axis=1)
    test_column = pollen_test.drop([column], axis=1)
    
    model = xgb.XGBRegressor()

    train_data = copy.deepcopy(pollen_train).drop([column], axis=1)
    model.fit(train_data, pollen_val[column])

    col_ans = model.predict(pollen_test.iloc[:, ~pollen_test.columns.isin([column, 'batch_id'])])

    if column == 'AMBROSIA':
        ans_columns['1 day prediction'] = [int(val) for val in col_ans]
        print(pollen_test[column][80:90], col_ans[80:90])

    pollen_test_copy[column] = (pollen_test_copy[column] + col_ans) / 2

pollen_test = copy.deepcopy(pollen_test_copy)

80    1.381250
81    0.900000
82    2.887500
83    1.856250
84    4.728125
85    6.971875
86    2.425000
87    3.293750
88    0.409375
89    6.659375
Name: AMBROSIA, dtype: float64 [1.8239978e-16 1.8239978e-16 1.8239978e-16 1.8239978e-16 1.8239978e-16
 1.8239978e-16 1.8239978e-16 1.8239978e-16 1.8239978e-16 1.8239978e-16]


In [328]:
pollen_test_copy = copy.deepcopy(pollen_test)

for idx, column in enumerate(columns):
    if column == 'date_idx':
        continue
    batch_id = pollen_test.drop(['batch_id'], axis=1)
    test_column = pollen_test.drop([column], axis=1)
    
    model = xgb.XGBRegressor()

    train_data = copy.deepcopy(pollen_train).drop([column], axis=1)
    model.fit(train_data, pollen_val[column])

    col_ans = model.predict(pollen_test.iloc[:, ~pollen_test.columns.isin([column, 'batch_id'])])

    if column == 'AMBROSIA':
        ans_columns['2 days prediction'] = [int(val) for val in col_ans]

    pollen_test_copy[column] = (pollen_test_copy[column] + col_ans) / 2

pollen_test = copy.deepcopy(pollen_test_copy)

In [329]:
pollen_test_copy = copy.deepcopy(pollen_test)

for idx, column in enumerate(columns):
    if column == 'date_idx':
        continue
    batch_id = pollen_test.drop(['batch_id'], axis=1)
    test_column = pollen_test.drop([column], axis=1)
    
    model = xgb.XGBRegressor()

    train_data = copy.deepcopy(pollen_train).drop([column], axis=1)
    model.fit(train_data, pollen_val[column])

    col_ans = model.predict(pollen_test.iloc[:, ~pollen_test.columns.isin([column, 'batch_id'])])

    if column == 'AMBROSIA':
        ans_columns['3 days prediction'] = [int(val) for val in col_ans]

    pollen_test_copy[column] = (pollen_test_copy[column] + col_ans) / 2

pollen_test = copy.deepcopy(pollen_test_copy)

In [330]:
ans_columns[70:90]

,batch_id,1 day prediction,2 days prediction,3 days prediction
70,71,0,0,0
71,72,0,0,0
72,73,0,0,0
73,74,0,0,0
74,75,0,0,0
75,76,0,0,0
76,77,0,0,0
77,78,0,0,0
78,79,0,0,0
79,80,0,0,0
